In [2]:
%pip install sagemaker

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
sess = boto3.Session()
sm = sess.client("sagemaker")
role = get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

pipeline_session = PipelineSession()

print(bucket)

sagemaker-ap-southeast-1-357080086340


In [5]:
DATASET_FILE = 'stock_price_consolidated.csv'

PROJECT_NAME = 'price-prediction'
DATA_PREFIX = f'project-work/{PROJECT_NAME}/data/' # S3 prefix to store data
MODEL_OUTPUT_S3_PATH = f's3://{bucket}/project-work/{PROJECT_NAME}/model/' # S3 prefix to store the XGBoost training information and model.

BASE_JOB_PROCESSING_NAME = f'{PROJECT_NAME}-processing'  # base_job_name for preprocessing
BASE_JOB_TRAINING_NAME = f'{PROJECT_NAME}-training'  # base_job_name for training
BASE_JOB_EVALUATION_NAME = f'{PROJECT_NAME}-evaluation'  # base_job_name for evaluation

PIPELINE_NAME = f'{PROJECT_NAME}-pipeline'  # SageMaker Pipeline name
MODEL_PACKAGE_GROUP_NAME = f'{PROJECT_NAME}-ModelPackageGroup'  # Model package group name in the Model Registry

print(f'DATA_PREFIX: {DATA_PREFIX}')
print(f'PIPELINE_NAME: {PIPELINE_NAME}')
print(f'MODEL_PACKAGE_GROUP_NAME: {MODEL_PACKAGE_GROUP_NAME}')

DATA_PREFIX: project-work/price-prediction/data/
PIPELINE_NAME: price-prediction-pipeline
MODEL_PACKAGE_GROUP_NAME: price-prediction-ModelPackageGroup


In [6]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat

# raw input data
input_data = ParameterString(name='InputData')

# status of newly trained model in registry
model_approval_status = ParameterString(name='ModelApprovalStatus', 
                                        default_value='Approved')

# processing step parameters
processing_instance_type = ParameterString(name='ProcessingInstanceType', 
                                           default_value='ml.m5.large')
processing_instance_count = ParameterInteger(name='ProcessingInstanceCount', 
                                             default_value=1)

# training step parameters
training_instance_type = ParameterString(name='TrainingInstanceType', 
                                         default_value='ml.m5.large')

# model performance step parameters
accuracy_rmse_threshold = ParameterFloat(name='ErrorThreshold', 
                                         default_value=10.0)

In [7]:
# raw_data = sagemaker_session.upload_data(path=DATASET_FILE,
#                                          key_prefix=DATA_PREFIX)
# raw_data

## Preprocessing Job

In [8]:
!mkdir -p code

In [9]:
%%writefile code/preprocess.py

import numpy as np
import pandas as pd
import os
import json
import joblib
from io import StringIO
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import tarfile

try:
    from sagemaker_containers.beta.framework import (
        content_types,
        encoders,
        env,
        modules,
        transformer,
        worker,
        server,
    )
except ImportError:
    pass

RANDOM_STATE = 2024
LABEL_COLUMN = 'Close_7_Days'


if __name__ == "__main__":
    df = pd.read_csv('/opt/ml/processing/input/stock_price_consolidated.csv')
    # df = pd.read_csv('stock_price_consolidated.csv')

    
    # Write Next 7 Days Data Python
    df.drop(LABEL_COLUMN, axis=1, inplace=True)
    df_7_days = df[['STOCK','Date','Close']]
    df_7_days['Date'] = (pd.to_datetime(df_7_days['Date'],dayfirst =True) - pd.Timedelta(days=7)).dt.strftime('%d/%m/%Y')
    df_7_days.rename(columns = {'Close':LABEL_COLUMN},inplace=True)
    # print(df_7_days.head(10))
    df = pd.merge(df,df_7_days,how = 'left', on = ['Date','STOCK'])
    df.dropna(inplace = True)
    
    
    
    df.drop('Date',axis=1, inplace=True)
    df.dropna(inplace=True)
    #df['Volume'] = df['Volume'].to_string(index=False)
    df['Volume'] = df['Volume'].str.replace(',','').astype(float)
    
    df = df.replace(['AAPL','AMD','AMZN','GOOG','META','MSFT','NVDA'],[0,1,2,3,4,5,6])
    
    # label_encoder = LabelEncoder()
    # df['STOCK']=label_encoder.fit_transform(df['STOCK'])

    
    
    # print(df.columns)
    label_data = df[LABEL_COLUMN]
    feature_data = df.drop(LABEL_COLUMN, axis=1, inplace=False)
    x_train, x_test, y_train, y_test = train_test_split(feature_data, label_data, 
                                                        test_size=0.2,
                                                        random_state=RANDOM_STATE)
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, 
                                                    test_size=0.5,
                                                    random_state=RANDOM_STATE)
    scaler = StandardScaler()
    scaler.fit(x_train)
    
    x_train = scaler.transform(x_train)
    x_val = scaler.transform(x_val)
    x_test = scaler.transform(x_test)

    train_df = pd.concat([y_train.reset_index(drop=True), pd.DataFrame(x_train)], axis=1)
    val_df = pd.concat([y_val.reset_index(drop=True), pd.DataFrame(x_val)], axis=1)
    test_df = pd.concat([y_test.reset_index(drop=True), pd.DataFrame(x_test)], axis=1)
    
    # train_df.to_csv('train.csv', header=True, index=False)
    # val_df.to_csv('validation.csv', header=True, index=False)
    # test_df.to_csv('test.csv', header=True, index=False)
    
    train_df.to_csv('/opt/ml/processing/train/train.csv', header=False, index=False)
    val_df.to_csv('/opt/ml/processing/validation/validation.csv', header=False, index=False)
    test_df.to_csv('/opt/ml/processing/test/test.csv', header=False, index=False)
    joblib.dump(scaler, 'model.joblib')
    with tarfile.open('/opt/ml/processing/scaler_model/model.tar.gz', 'w:gz') as tar_handle:
        tar_handle.add('model.joblib')
    
    


def input_fn(input_data, content_type):
    """Parse input data payload.
    We currently only take csv input. 
    """
    if content_type == 'text/csv':
        # Read the raw input data as CSV.
        df = pd.read_csv(StringIO(input_data), header=None)
        return df
    else:
        raise ValueError(f'{content_type} not supported by the script!')


def output_fn(prediction, accept):
    """Format prediction output
    """
    accept = "text/csv"
    if accept == "application/json":
        print("preprocessing output as application/json")
        instances = []
        for row in prediction.tolist():
            instances.append(row)
        json_output = {"instances": instances}

        return worker.Response(json.dumps(json_output), mimetype=accept)
    elif accept == "text/csv":
        print("preprocessing output as text/csv")
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException(f'{accept} accept type is not supported by this script.')


def predict_fn(input_data, model):
    """Preprocess input data
    We implement this because the default predict_fn uses .predict(), but our model is a preprocessor
    so we want to use .transform().
    """
    features = model.transform(input_data)
    return features


def model_fn(model_dir):
    """Deserialize fitted StandardScaler model"""
    preprocessor = joblib.load(os.path.join(model_dir, 'model.joblib'))
    return preprocessor

Overwriting code/preprocess.py


In [10]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput


sklearn_processor = SKLearnProcessor(
    framework_version='1.2-1',
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name=BASE_JOB_PROCESSING_NAME,
    role=role,
    sagemaker_session=pipeline_session,
)

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination='/opt/ml/processing/input'),
    ],
    outputs=[
        ProcessingOutput(output_name='scaler_model', source='/opt/ml/processing/scaler_model'),
        ProcessingOutput(output_name='train', source='/opt/ml/processing/train'),
        ProcessingOutput(output_name='validation', source='/opt/ml/processing/validation'),
        ProcessingOutput(output_name='test', source='/opt/ml/processing/test'),
    ],
    code='code/preprocess.py',
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [11]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


step_process = ProcessingStep(
    name='PreprocessData',
    step_args=processor_args,
)

## Training Step

In [12]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


image_uri = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.7-1'
)

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=MODEL_OUTPUT_S3_PATH,
    base_job_name=BASE_JOB_TRAINING_NAME,
    sagemaker_session=pipeline_session,
    role=role,
)

xgb_train.set_hyperparameters(
    objective='reg:squarederror',
    num_round=50,
    max_depth=4,
    eta=0.2,
    gamma=4,
    reg_lambda=10,
    min_child_weight=6,
    subsample=0.7,
    verbosity=0,
)

train_args = xgb_train.fit(
    inputs={
        'train': TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs['train'].S3Output.S3Uri,
            content_type='text/csv',
        ),
        'validation': TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                'validation'
            ].S3Output.S3Uri,
            content_type='text/csv',
        ),
    },
)

step_train_model = TrainingStep(
    name='TrainMLModel',
    step_args=train_args
)

## Model Evaluation Step

In [13]:
%%writefile code/evaluate.py

"""Evaluation script for measuring accuracy."""
import json
import logging
import math
import pickle
import tarfile

import numpy as np
import pandas as pd
import xgboost

from pathlib import Path

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())


if __name__ == "__main__":
    logger.debug("Starting evaluation.")
    model_path = "/opt/ml/processing/model/model.tar.gz"
    logger.info(model_path)
    logger.info(Path(model_path).exists())
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")

    logger.debug("Loading xgboost model.")
    model = xgboost.Booster()
    model.load_model("xgboost-model")

    
    logger.debug("Reading test data.")
    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)

    logger.debug("Reading test data.")
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = xgboost.DMatrix(df.values)

    logger.info("Performing predictions against test data.")
    predictions = model.predict(X_test)

    logger.debug("Calculating accuracy.")
    
    mse = np.square(np.subtract(y_test, predictions)).mean() 
    rmse = math.sqrt(mse)
    report_dict = {
        "regression_metrics": {
            "rmse": {"value": rmse, "standard_deviation": "NaN"},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    logger.info(f'Writing out evaluation report with rmse: {rmse}')
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting code/evaluate.py


In [14]:

from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name=BASE_JOB_EVALUATION_NAME,
    sagemaker_session=pipeline_session,
    role=role,
)

evaluation_report = PropertyFile(
    name="A2EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
        ),
    ],
    code="code/evaluate.py",
)
step_evaluate_model = ProcessingStep(
    name="EvaluateMLModel",
    step_args=eval_args,
    property_files=[evaluation_report]
)

## Model step

In [15]:
# Model Package Creation

from sagemaker import Model
from sagemaker.sklearn import SKLearnModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.xgboost import XGBoostPredictor
from sagemaker.pipeline import PipelineModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.functions import Join


sklearn_model = SKLearnModel(
   model_data=Join(
       on='/',
       values=[step_process.properties.ProcessingOutputConfig.Outputs["scaler_model"].S3Output.S3Uri, 
               'model.tar.gz']
   ),
   entry_point='preprocess.py', 
   source_dir='code/', 
   framework_version='1.2-1',
   role=role,
   sagemaker_session=pipeline_session,
   py_version='py3'
)

xgboost_model = Model(
    image_uri=image_uri,
    model_data=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
    predictor_cls=XGBoostPredictor,
)

pipeline_model = PipelineModel(
    models=[sklearn_model, xgboost_model],
    role=role,
    sagemaker_session=pipeline_session
)

s3_eval_path = step_evaluate_model.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
evaluation_s3_uri = f"{s3_eval_path}/evaluation.json"

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

register_model_step_args = pipeline_model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.large"],
    #transform_instances=["ml.m5.large"],
    model_package_group_name=MODEL_PACKAGE_GROUP_NAME,
    model_metrics=model_metrics,
    approval_status=model_approval_status
)

step_create_model = ModelStep(
    name="CreateMLModel",
    step_args=register_model_step_args,
)

## Condition Step RSME

In [16]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_evaluate_model.name,
        property_file=evaluation_report,
        json_path="regression_metrics.rmse.value",
    ),
    right=accuracy_rmse_threshold,
)

step_cond = ConditionStep(
    name="CheckMLEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model],
    else_steps=[],
)

## Pipeline Creation

In [17]:
from sagemaker.workflow.pipeline import Pipeline

# Create a Sagemaker Pipeline.
pipeline = Pipeline(
    name=PIPELINE_NAME,
    parameters=[
        training_instance_type,
        processing_instance_type,
        processing_instance_count,
        input_data,
        model_approval_status,
        accuracy_rmse_threshold,
    ],
    steps=[step_process, 
           step_train_model,
           step_evaluate_model, 
           step_cond
          ],
)

In [18]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InputData', 'Type': 'String'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'Approved'},
  {'Name': 'ErrorThreshold', 'Type': 'Float', 'DefaultValue': 10.0}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'PreprocessData',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.ProcessingInstanceType'},
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '121021644041.dkr.ec

In [19]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:357080086340:pipeline/price-prediction-pipeline',
 'ResponseMetadata': {'RequestId': '5e0d0323-6da1-4d36-a3de-0ff359c79b7e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5e0d0323-6da1-4d36-a3de-0ff359c79b7e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Sun, 30 Jun 2024 11:43:31 GMT'},
  'RetryAttempts': 0}}

In [21]:
# Raw Data Location Update to S3 Loc

raw_data = "s3://webscrape-bucket-mle611/stock_price_consolidated.csv"

execution = pipeline.start(
    parameters=dict(
        InputData=raw_data,
    )
)

In [22]:
execution.wait()

## Deploy And Test Model

In [23]:
ENDPOINT_NAME = f'{PROJECT_NAME}-endpoint-' + time.strftime('%Y-%m-%d-%H-%M-%S', time.gmtime())
print(f'Endpoint Name= {ENDPOINT_NAME}')

Endpoint Name= price-prediction-endpoint-2024-06-30-11-53-50


In [24]:
from botocore.exceptions import ClientError

sm_client = boto3.client("sagemaker")

def get_approved_package(model_package_group_name):
    """Gets the latest approved model package for a model package group.

    Args:
        model_package_group_name: The model package group name.

    Returns:
        The SageMaker Model Package ARN.
    """
    try:
        # Get the latest approved model package
        response = sm_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # Fetch more packages if none returned with continuation token
        while len(approved_packages) == 0 and "NextToken" in response:
            response = sm_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        # Return error if no packages found
        if len(approved_packages) == 0:
            error_message = (
                f"No approved ModelPackage found for ModelPackageGroup: {model_package_group_name}"
            )
            raise Exception(error_message)

        # Return the pmodel package arn
        model_package_arn = approved_packages[0]["ModelPackageArn"]
        return approved_packages[0]
        # return model_package_arn
    except ClientError as e:
        error_message = e.response["Error"]["Message"]
        raise Exception(error_message)

In [25]:
pck = get_approved_package(
    MODEL_PACKAGE_GROUP_NAME
)
model_description = sm_client.describe_model_package(ModelPackageName=pck["ModelPackageArn"])

model_description

{'ModelPackageGroupName': 'price-prediction-ModelPackageGroup',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-southeast-1:357080086340:model-package/price-prediction-ModelPackageGroup/2',
 'CreationTime': datetime.datetime(2024, 6, 30, 11, 53, 20, 76000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '121021644041.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3',
    'ImageDigest': 'sha256:ed242e33af079f334972acd2a7ddf74d13310d3c9a0ef3a0e9b0429ccc104dcd',
    'ModelDataUrl': 's3://sagemaker-ap-southeast-1-357080086340/price-prediction-pipeline/turooilalsvi/PreprocessData/output/scaler_model/model.tar.gz',
    'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
     'SAGEMAKER_PROGRAM': 'preprocess.py',
     'SAGEMAKER_REGION': 'ap-southeast-1',
     'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-ap-southeast-1-357080086340/sagemaker-scikit-learn-2024-06-30-11-43-30-120/sourcedir.tar.gz'}},
   {'Image': '1210216

In [26]:
from sagemaker import ModelPackage

model_package_arn = model_description["ModelPackageArn"]
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

model.deploy(initial_instance_count=1, instance_type="ml.m5.large", endpoint_name=ENDPOINT_NAME)

INFO:sagemaker:Creating model with name: price-prediction-ModelPackageGroup-2024-06-30-11-53-50-785
INFO:sagemaker:Creating endpoint-config with name price-prediction-endpoint-2024-06-30-11-53-50
INFO:sagemaker:Creating endpoint with name price-prediction-endpoint-2024-06-30-11-53-50


---------!

In [27]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name=ENDPOINT_NAME)

In [28]:
# # Please put new Test File into Local, we are using it for the testing job

# # If ok, change the lambda job to this endpoint

# LABEL_COLUMN = 'Close_7_Days'

# df = pd.read_csv('test.csv')
# label_data = df[LABEL_COLUMN]
# feature_data = df.drop(LABEL_COLUMN, axis=1, inplace=False)

# payload = feature_data.to_csv(header=False, index=False)

# p = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
# print(p.decode("utf-8"))

100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625
100.0211181640625



## Clean-up
In case we need it. DO NO RUN BEYOND THIS POINT

### Delete model package

In [22]:
# sm_client = boto3.client("sagemaker")

# for d in sm_client.list_model_packages(ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME)[
#     "ModelPackageSummaryList"
# ]:
#     print(d["ModelPackageArn"])
#     sm_client.delete_model_package(ModelPackageName=d["ModelPackageArn"])

# sm_client.delete_model_package_group(ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME)

arn:aws:sagemaker:ap-southeast-1:357080086340:model-package/price-prediction-ModelPackageGroup/1


{'ResponseMetadata': {'RequestId': '4a6e6fea-786f-4602-af9c-eded555833dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4a6e6fea-786f-4602-af9c-eded555833dc',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 30 Jun 2024 05:32:20 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

### Delete endpoint

In [23]:
# predictor.delete_endpoint()

NameError: name 'predictor' is not defined

### Delete pipeline

In [24]:
# pipeline.delete()

INFO:sagemaker.workflow.pipeline:If triggers have been setup for this target, they will become orphaned.You will need to clean them up manually via the CLI or EventBridge console.


{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:357080086340:pipeline/Price-Prediction-pipeline',
 'ResponseMetadata': {'RequestId': 'fec18b54-1cb7-4492-8bf3-e8c8a3541d84',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fec18b54-1cb7-4492-8bf3-e8c8a3541d84',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Sun, 30 Jun 2024 05:32:26 GMT'},
  'RetryAttempts': 0}}